In [379]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import json
from rauth import OAuth2Service
from lyricsgenius import Genius
from clean_colored_lyrics import *
from synced_lyrics import *
from fuzzysearch import find_near_matches
from datetime import datetime, timedelta, date
import pandas as pd
import math
import os
from pathlib import Path
import copy
ACCESS_TOKEN = "WgL1MoH7WJNIR-EdDwj2zhga4YUYwW4ipFOXRIvqn7hVQh6ZLLBhUA4qZr5SypIu"
genius = Genius(ACCESS_TOKEN)

In [380]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup = soup.body
    return soup
    

In [381]:
with open('../json/raw_albums.json', 'r') as file:
    all_urls = json.load(file)


In [382]:
def get_lyrics(raw_html, debug):
    lyrics_container = raw_html.findAll('div', class_='Lyrics__Container-sc-68a46031-1 ibbPVY')

    found_section = False
    section_style = []
    assigned_lyrics = []
    members = []
    section = ''
    line = 0
    style_start = False

    for section in lyrics_container:
        if debug: print(section)
        for l in section:
            if l.name == 'div':
                continue
            
            if l.name == 'br':
                if debug: print('br')
                line += 1
                continue
            
            l_style = [l.name]
            l_text = l.get_text()
            if l_text.find('*') >= 0:
                l_style = ['*']
                if l_text.find('**') >= 0:
                    l_style = ['**']

            if l_text == '' or l_text.strip() == ',':
                continue

            if l_text.find('English Translation') >= 0:
                break
            
            if not l.name == None:
                for x in l.contents:
                    if not x.name == None and (x.name == 'i' or x.name == 'b'):
                        if debug: print("CHILDREN", x, x.name)
                        l_style.append(x.name)
                l_text = l.get_text(separator="\n", strip=True)
            
            if debug: print(l_style, l_text, found_section)

            if not found_section:
                if l_text.find('[') >= 0:
                    section_style = []
                    section = ''
                    line = 0
                    found_section = True

                    if l_text.find(':') == -1:
                        l_text = l_text.replace('[', '').replace(']', '')
                        section = l_text
                        section_style.append({'style': [None], 'member': ['None']})
                        if debug: print('new me', {'style': [None], 'member': ['None']})
                        found_section = False
                        continue

                    if l_text.find(']') >= 0:
                        found_section = False
                        l_text = l_text[:l_text.find(']')]
                    
                    section = l_text[1:l_text.find(':')]

                    member = l_text[l_text.find(':')+1:].replace(',','').strip()
                
                    if '8' in member:
                        i = member.find('8')
                        if i >= 0:
                            member = 'The8'
                    
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members:
                            members.append(m)

                else:
                    curr_member = next((style['member'] for style in section_style if set(style['style']) == set(l_style)), None)
                    
                    if l_style[0] == '*' or l_style[0] == '**':
                        l_text = l_text.replace(l_style[0], '')

                    l_text = l_text.replace('(', '').replace(')', '').strip()
                    
                    while not l_text == '':
                        i = l_text.find('\n')
                        curr_t = l_text[:i]
                        l_text = l_text[i+1:]
                        if normalize_lyrics(curr_t).replace(',', '').strip() == '':
                            l_text == ''
                            break

                        if i == -1:
                            if debug: print('ADD LINE ---> ',{'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            assigned_lyrics.append({'member': curr_member, 'lyric': l_text, 'section': section, 'line': line})
                            break

                        if debug: print('ADD LINE ---> ', normalize_lyrics(curr_t), {'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        assigned_lyrics.append({'member': curr_member, 'lyric': curr_t, 'section': section, 'line': line})
                        line += 1
            else:
                if l_text == ']':
                    found_section = False
                    continue

                if l_text.find(']') >= 0:
                    found_section = False
                    l_text = l_text[:l_text.find(']')]

                if debug: print('found', found_section, l_text)
                
                if l_text.find('*') >= 0:
                    pattern = r'(\*\*|\*)([^*]+)\1'
                    for marker, name in re.findall(pattern, l_text):
                        l_style = "**" if marker == "**" else "*"
                        pattern = fr'{re.escape(l_style)}(.*?){re.escape(l_style)}'
                        member = re.findall(pattern, l_text)[0]

                        if '8' in member:
                            i = member.find('8')
                            if i >= 0:
                                member = 'The8'
                        member = re.split(r'[&,\s]+', member)
                        section_style.append({'style': [l_style], 'member': member})
                        if debug: print('new me', {'style': [l_style], 'member': member}, name)

                        for m in member:
                            if m not in members:
                                members.append(m)
                else:
                    member = l_text[l_text.find(':')+1:].replace(',','').strip()

                    if '8' in member:
                        i = member.find('8')
                        if i >= 0:
                            member = 'The8'
                    member = re.split(r'[&,\s]+', member)
                    if debug: print('new me', {'style': l_style, 'member': member})
                    section_style.append({'style': l_style, 'member': member})

                    for m in member:
                        if m not in members:
                            members.append(m)

            if debug: print()

        if debug: print()

    return {'assigned_lyrics': assigned_lyrics, "members": members}

# results = get_lyrics(raw_html, True)

In [383]:
def merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    c_i = 0
    s_i = 0
    m_i = 0

    # colored_lyrics_copy = copy.deepcopy(colored_lyrics[:20])
    # synced_lyrics_copy = copy.deepcopy(synced_lyrics[:20])
    
    colored_lyrics_copy = copy.deepcopy(colored_lyrics)
    synced_lyrics_copy = copy.deepcopy(synced_lyrics)

    merged_lines = copy.deepcopy(colored_lyrics_copy)
    for m in merged_lines:
        m['lyric'] = m['lyric'].strip()
        m['start'] = ''
        m['end'] = ''
        m['delta'] = 0

    unmerged_line_color = []
    unmerged_line_synced = []

    c_looking_ahead_index = 0
    s_looking_ahead_index = 0

    c_len_shorter = True
    check_color = True

    while c_i < len(colored_lyrics) or s_i < len(synced_lyrics_copy):
        if debug: print('c_i',c_i, '  s_i', s_i, '  m_i', m_i, '  c_looking_ahead_index', c_looking_ahead_index, '    s_looking_ahead_index', s_looking_ahead_index)

        if c_i >= len(colored_lyrics) or c_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy) or s_looking_ahead_index > looking_ahead_max:
            if debug: print('MERGE REMAINING COLOR')
            for c in colored_lyrics[c_i:]:
                unmerged_line_color.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break


        if c_i + c_looking_ahead_index >= len(colored_lyrics):
            if debug: print("OVER COLOR")
            looking_ahead_max = max(0, len(merged_lines) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break
        elif s_i + s_looking_ahead_index >= len(synced_lyrics_copy):
            if debug: print("OVER SYNCED")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            break

        c_line = colored_lyrics_copy[c_i + c_looking_ahead_index]
        s_line = synced_lyrics_copy[s_i + s_looking_ahead_index]
        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        m_line = merged_lines[m_i]
        if debug: print(c_line, s_line, m_line)
        if debug: print(c_lyric_norm, s_lyric_norm)

        max_l_dist = 5
        c_len = len(c_lyric_norm)
        s_len = len(s_lyric_norm)
        if c_len < 5 or s_len < 5:
            max_l_dist = 1
        elif c_len < 10 or s_len < 10:
            max_l_dist = int(min(c_len, s_len) / 3)

        find_match = []
        remaining_match = []
        remaining_str = ''
        if c_len <= s_len:
            c_len_shorter = True
            find_match = find_near_matches(c_lyric_norm, s_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = s_lyric_norm
            if debug: print('c_len find_match: ', find_match)
        else:
            c_len_shorter = False
            find_match = find_near_matches(s_lyric_norm, c_lyric_norm, max_l_dist=max_l_dist)
            remaining_str = c_lyric_norm
            if debug: print('s_len find_match: ', find_match)

        if debug: print('found', find_match)
        found = False
        match_end = 0

        if find_match:
            found = find_match[0].start <= 6 and find_match[0].matched
            match_end = find_match[0].end
            find_match = found
        
        if not find_match:
            found = set(c_lyric_norm) == set(s_lyric_norm)
            if debug: print('SAME SET OF LETTERS', found, set(c_lyric_norm), set(s_lyric_norm))
            match_end = min(len(c_lyric), len(s_lyric))
            
            if not found:
                prefix = os.path.commonprefix([c_lyric, s_lyric])
                if debug: print('PREFIX: ' ,prefix)
                if len(prefix) >= 1: 
                    if debug: print('STARTS THE SAME')
                    match_end = len(prefix)
                    found = True
            

        remaining_str = remaining_str[match_end:].strip()
        if debug: print("REMINAING STR:" , remaining_str)

        if found:
            for i in range(s_looking_ahead_index):
                new_line = {
                    'member': ['adlib'],
                    'lyric': synced_lyrics_copy[s_i+i]['lyric'],
                    'start': synced_lyrics_copy[s_i+i]['start'],
                    'end': synced_lyrics_copy[s_i+i]['end'],
                    'delta': synced_lyrics_copy[s_i+i]['delta'],
                    'section': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['section'],
                    'line': colored_lyrics_copy[max(0,c_i + c_looking_ahead_index - 1)]['line'],
                    # 'section_name': ''
                }
                merged_lines.insert(m_i + i, new_line)
                if debug: print("INSERT SYNCED MERGE --->", m_i + i, new_line )
                m_i += 1

            if c_looking_ahead_index and not m_line['lyric'] == c_line['lyric']:
                if debug: print('SKIPPING COLOR -->', m_i, m_line)
                m_i += c_looking_ahead_index

            m_line = merged_lines[m_i]
            if debug: print('NEW M:', m_i , m_line)

            if c_len_shorter and not remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)
                if debug: print('EXACT COLOR MERGE --->', m_i, m_line)
                c_i += (1 + c_looking_ahead_index)
                s_i += (1 + s_looking_ahead_index)
                m_i += 1
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0

            if not c_len_shorter and not remaining_str:
                m_line = merged_lines[m_i]
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                m_line['end'] = s_line['end']
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                if debug: print('EXACT SYNCED MERGE --->', m_i, m_line)
                c_i += 1
                s_i += (1 + s_looking_ahead_index)
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0
                check_color = True
                m_i += 1

            if c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']

                ratio = float(match_end) / float(len(s_lyric_norm))
                d = round(s_line['delta'] * ratio, 2)

                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_start = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=d)).time())
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)

                if new_start.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_start = re.findall(pattern, new_start)[0]
                if debug: print('new_start', s_line['start'], (datetime.combine(datetime.now(), time_object)), new_start, d)
                s_line['start'] = new_start
                s_line['delta'] = round(s_line['delta'] - d, 2)
                s_line['lyric'] = remaining_str

                m_line['delta'] = round(m_line['delta'] + d, 2)
                m_line['end'] = new_start
                check_color = True
                                
                c_i += (1 + c_looking_ahead_index)
                s_i += s_looking_ahead_index
                s_looking_ahead_index = 0
                c_looking_ahead_index = 0

                if debug: print('PARTIAL S_LINE MERGE --->', m_i, m_line)
                m_i += 1

            if not c_len_shorter and remaining_str:
                if m_line['start'] == '':
                    m_line['start'] = s_line['start']
                c_line['lyric'] = remaining_str
                m_line['delta'] = round(m_line['delta'] + s_line['delta'], 2)

                time_object = datetime.strptime(s_line['start'], '%M:%S:%f').time()
                new_end = str(((datetime.combine(datetime.now(), time_object)) + timedelta(seconds=m_line['delta'])).time())
                if new_end.find('.') > 0:
                    pattern = r"\d{2}:(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end.replace('.', ':'))[0]
                else:
                    pattern = r"(\d{2}:\d{2}:\d{2})"
                    new_end = re.findall(pattern, new_end)[0]

                m_line['end'] = new_end

                s_i += (1 + s_looking_ahead_index)
                c_i += c_looking_ahead_index
                c_looking_ahead_index = 0
                s_looking_ahead_index = 0
                check_color = True

                if debug: print('PARTIAL C_LINE MERGE --->', m_i, m_line)
                # m_i += 1

        else:
            if check_color:
                c_looking_ahead_index += 1
                s_looking_ahead_index = 0
                if debug: print("CHECK COLOR")
            else: 
                s_looking_ahead_index += 1
                c_looking_ahead_index = 0
                if debug: print("CHECK SYNCED")

        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            if not check_color:
                # s_i += 1
                c_i += 1

            new_line = {
                'member': '',
                'lyric': s_line['lyric'],
                'start': s_line['start'],
                'end': s_line['end'],
                'delta': s_line['delta'],
                'section': '',
                'line': '',
                # 'section_name': ''
            }
            merged_lines.insert(m_i, new_line)
            check_color = True
            if debug: print("reset all INSERT MERGE --->", m_i, new_line )
            m_i += 1

        if c_looking_ahead_index > looking_ahead_max:
            c_looking_ahead_index = 0
            if not check_color:
                c_i += 1
            else:
                check_color = False
            if debug: print("c_looking_ahead_index reset")
        
        if s_looking_ahead_index > looking_ahead_max:
            s_looking_ahead_index = 0
            s_i += 1
            if debug: print("s_looking_ahead_index reset")

        if debug: print()

    key_order = ['start', 'end', 'delta', 'section_name', 'section', 'line', 'member', 'lyric'] 
    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]

    return merged_lines


In [ ]:
def update_txt(album_name, track):
    with open('../json/raw_txt/'+ album_name.replace(' ', '_')+'_discography.txt', 'a') as file:
        file.write('\n')
        file.write('NEW UPDATE')
        file.write('\n')
        json.dump(track, file, ensure_ascii=False)
        file.write('\n')
        for t in track['synced_lyrics']:
            json.dump(t, file, ensure_ascii=False)
            file.write('\n')
        file.write('\n')
        file.write('\n')
    print('wrote to: ../json/raw_txt/'+ album_name.replace(' ', '_')+'_discography.txt')

In [ ]:
def get_track_lyrics(album_tracks, album_name, merge_debug, get_lyrics_debug, look_ahead_max):
    tracks = []
    create_track = {}

    for track in album_tracks:
        song = track['song']
        song = genius.search_song(song['title'], song['artist'])
        
        if song is None:
            print('genius did not find')
            print()
            continue
        song = song.to_dict()
        print(song)

        english_name = re.sub(r'[^a-zA-Z0-9\s.,!?\'"*-]', '', song['title']).strip()
        print("english_name" , english_name)

        raw_html = get_soup(song['url'])
        print('get lyrics')
        lyrics_results = get_lyrics(raw_html, get_lyrics_debug)
        
        meta_data = get_song_info(english_name, 'seventeen', album_name)

        if meta_data == 'song not found':
            print()
            continue
        
        synced_lyrics = meta_data['syncedLyrics']
        synced_lyrics = merge_singular(lyrics_results['assigned_lyrics'], synced_lyrics, look_ahead_max, merge_debug)
        curr_members = lyrics_results['members']

        ot13 = ['S. Coups', 'Jeonghan', 'Joshua',  'Jun', 'Hoshi', 'Wonwoo', 'Woozi' , 'The8', 'Mingyu', 'DK', 'Seungkwan', 'Vernon', 'Dino', 'DJ Khalid']
        ot13_norm = {
            normalize_lyrics(name): name
            for name in ot13
        }
        found_members = []
        for c in curr_members:
            c_norm = normalize_lyrics(c)
            if c_norm in ot13_norm:
                found_members.append(ot13_norm[c_norm])

        order = {name: i for i, name in enumerate(ot13)}
        sorted_members = sorted(found_members, key=lambda x: order[x])

        create_track = {
            "track_name": song['title'],
            "english_name": english_name,
            "url": song['url'],
            "duration": meta_data['duration'],
            "featured_artists": sorted_members,
            "unit": get_unit(sorted_members),
            "media": song['media'],
            "lyrics": lyrics_results['assigned_lyrics'],
            "synced_lyrics": synced_lyrics,
            "synced_lyrics_source": meta_data['source'],
            "custom_performances": song['custom_performances']
            }
        tracks.append(create_track)
        print(create_track)
        update_txt(album_name, create_track)
        print('done')
        print()
    return tracks
    

In [ ]:
discography = []
for url in all_urls[10:11]:
    raw_html = get_soup(url['url'])
    album_type = url['album_type']
    print(url)
    header = raw_html.find('h1').text
    print(header)

    song_section = raw_html.findAll("a", class_="u-display_block")
    titles = []
    for s in song_section:
        title = s.find('h3', class_="chart_row-content-title", recursive=False).text.strip()
        title = title[:title.find('\n')]
        url = s["href"]
        print({'song_name': title, 'url': url})
        titles.append({'song_name': title, 'url': url})

    release = raw_html.find('div', class_='metadata_unit').text
    release = release[len('release')+1:].strip()
 
    album = genius.search_album(header, "seventeen")
    dict_album = album.to_dict()

    disco = [{
        "album_name": dict_album['name'],
        "artist": dict_album['artist'],
        "album_type": album_type,
        "cover_art_url": dict_album['cover_art_url'],
        "release_date_components": dict_album['release_date_components'],
        "release_date_for_display": dict_album['release_date_for_display'],
        "tracks": get_track_lyrics(dict_album['tracks'][3:4], dict_album['name'], 0, 0, 4)
    }]

    print(album)
    discography.append(album)
    

{'url': 'https://genius.com/albums/Seventeen/Fml', 'album_type': 'EP'}
FML
{'song_name': 'F*ck My Life', 'url': 'https://genius.com/Seventeen-fck-my-life-lyrics'}
{'song_name': '손오공 (Super)', 'url': 'https://genius.com/Seventeen-super-lyrics'}
{'song_name': 'Fire', 'url': 'https://genius.com/Seventeen-fire-lyrics'}
{'song_name': "I Don't Understand But I Luv U", 'url': 'https://genius.com/Seventeen-i-dont-understand-but-i-luv-u-lyrics'}
{'song_name': '먼지 (Dust)', 'url': 'https://genius.com/Seventeen-dust-lyrics'}
{'song_name': 'April shower', 'url': 'https://genius.com/Seventeen-april-shower-lyrics'}
Searching for "FML" by seventeen...
Searching for "I Don’t Understand But I Luv U" by SEVENTEEN (세븐틴)...
Done.
{'_type': 'song', 'annotation_count': 2, 'api_path': '/songs/8978810', 'artist_names': 'SEVENTEEN (세븐틴)', 'full_title': "I Don't Understand But I Luv U by\xa0SEVENTEEN\xa0(세븐틴)", 'header_image_thumbnail_url': 'https://images.genius.com/e653b69af606b96c3cc3a7d6e94526c0.300x300x1.pn

In [ ]:
disco = []
for a in discography[:1]:
    album = genius.search_album(a['album_name'], "seventeen")
    dict_album = album.to_dict()

    disco = [{
        "album_name": dict_album['name'],
        "artist": dict_album['artist'],
        "album_type": a['album_type'],
        "cover_art_url": dict_album['cover_art_url'],
        "release_date_components": dict_album['release_date_components'],
        "release_date_for_display": dict_album['release_date_for_display'],
        "tracks": get_track_lyrics(dict_album['tracks'][1:2], dict_album['name'], 1,1, 4)
    }]

    print()

Searching for "17 CARAT" by seventeen...
Searching for "아낀다 (Adore U)" by SEVENTEEN (세븐틴)...


Timeout: Request timed out:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)

In [ ]:
def update(time, song):
    diff = 0
    for t in song:
        new_start = time_to_sec(time)
        if not t['start'] == '' and not diff:
            print(t)
            old_start = time_to_sec(t['start'])
            diff = new_start - old_start
            print("diff", diff)

            # mv_start = time_to_sec(mv_synced[0]['start'])
            # lib_start = time_to_sec(meta_data['syncedLyrics'][0]['start'])
            # dif = mv_start - lib_start

        if not t['start'] == '':
            print(t)
            old_start = time_to_sec(t['start']) + diff
            old_end = time_to_sec(t['end']) + diff
            # print(decimal_sec_to_time_str(old_start),decimal_sec_to_time_str(old_end))
            t['start'] = decimal_sec_to_time_str(old_start)
            t['end'] = decimal_sec_to_time_str(old_end)

# "00:14:93"
# disco[0]['tracks'][0]['synced_lyrics']
# update_txt('FML', disco[0]['tracks'][0:])

wrote to: ../json/raw_txt/FML_discography.txt


In [ ]:
with open('../json/raw_txt/'+a['album_name'].replace(' ', '_')+'_discography.txt', 'w') as file:
    for d in disco:
        json.dump(d, file, ensure_ascii=False)
        file.write('\n')
        for track in d['tracks']:
            json.dump(track, file, ensure_ascii=False)
            file.write('\n')
            for l in track['synced_lyrics']:
                json.dump(l, file, ensure_ascii=False)
                file.write('\n')
            file.write('\n')
            file.write('\n')
        file.write('\n')
        file.write('\n')

In [442]:
final_disco = {}
tracks = []
lines = []

curr_album = []
curr_track = []

album = 'YOU MADE MY DAWN'

with open('../json/raw_txt/'+ album.replace(' ', '_') + '_discography.txt', 'r') as file:
    for l in file:
        l = l.strip()
        if l:
            if l == 'NEW UPDATE':
                print('new update')
                print()
                continue
        
            obj = json.loads(l)
            print(obj)
            if obj.get("album_name") is not None:
                obj['norm_name'] = normalize_lyrics(obj.get('album_name'))
                obj["album_type"] = "EP"
                obj['tracks'] = []
                final_disco = obj
                # final_disco.append(obj) 
                curr_album = obj
            
            if obj.get("synced_lyrics")  is not None:
                obj['synced_lyrics'] = []
                obj['norm_name'] = normalize_lyrics(obj.get('track_name'))
                curr_album['tracks'].append(obj)
                curr_track = obj

            if obj.get('start') is not None:
                if obj['start'] == '':
                    obj['is_adlib'] = True
                else:
                    obj['delta'] = getDelta(obj, ':')
                    for i, m in enumerate(obj['member']):
                        obj['member'][i] = obj['member'][i].replace('(','').replace(')','')
                        if m.lower().find('coups') >= 0:
                            obj['member'][i] = 'S. Coups'
                        elif m.find('8') >= 0:
                            obj['member'][i] = 'The8'
                        elif m.lower() == 'dk':
                            obj['member'][i] = 'DK'
                        else:
                            obj['member'][i] = obj['member'][i][:1].upper() + obj['member'][i][1:].lower()
                            
                        if 'adlib' in m.lower():
                            obj['member'][i] = 'adlib'
                            obj['is_adlib'] = True
                        else:
                            obj['is_adlib'] = False
                curr_track['synced_lyrics'].append(obj)

new update

{'album_name': 'YOU MADE MY DAWN', 'artist': 'SEVENTEEN (세븐틴)', 'cover_art_url': 'https://images.genius.com/158c9f6890076bc55bb36607b0178ceb.1000x1000x1.png', 'release_date_components': {'year': 2019, 'month': 1, 'day': 21}, 'release_date_for_display': 'January 21, 2019', 'tracks': [{'track_name': 'Good to Me', 'english_name': 'Good to Me', 'url': 'https://genius.com/Seventeen-good-to-me-lyrics', 'duration': 190.0, 'featured_artists': ['S. Coups', 'Jeonghan', 'Joshua', 'Jun', 'Hoshi', 'Wonwoo', 'Woozi', 'The8', 'Mingyu', 'DK', 'Seungkwan', 'Vernon', 'Dino'], 'unit': ['OT13'], 'media': [{'provider': 'youtube', 'start': 0, 'type': 'video', 'url': 'https://www.youtube.com/watch?v=aFiGhsY1vcE'}], 'lyrics': [{'member': ['DK'], 'lyric': '한 번쯤 그런 적 있잖아', 'section': 'Verse 1', 'line': 1}, {'member': ['DK'], 'lyric': '눈만 봐도 다 알 수가 있는', 'section': 'Verse 1', 'line': 2}, {'member': None, 'lyric': 'Woah, yeah', 'section': 'Verse 1', 'line': 2}, {'member': ['Seungkwan'], 'lyric': '뭘 어쩌나

In [ ]:

for t in final_disco['tracks']:
    print(t)
    print(t['norm_name'])
    for l in t['synced_lyrics']:
        print(l)
    print()


{'track_name': '표정관리 (Fronting)', 'english_name': 'Fronting', 'url': 'https://genius.com/Seventeen-fronting-lyrics', 'duration': 185.0, 'featured_artists': ['S. Coups', 'Hoshi', 'Wonwoo', 'Woozi', 'Mingyu', 'Vernon'], 'unit': ['Hoshi', 'Mingyu', 'S. Coups', 'Vernon', 'Wonwoo', 'Woozi'], 'media': [{'provider': 'youtube', 'start': 0, 'type': 'video', 'url': 'https://www.youtube.com/watch?v=45N_RlRVKKI'}], 'lyrics': [{'member': ['Hoshi'], 'lyric': 'I woke up 꿈마저도, baby', 'section': 'Chorus', 'line': 1}, {'member': ['Hoshi'], 'lyric': '널 따라 다니는 꿈꿔 어떡해', 'section': 'Chorus', 'line': 2}, {'member': ['Hoshi'], 'lyric': '다른 소녀들 너만큼은 안돼', 'section': 'Chorus', 'line': 3}, {'member': ['Hoshi'], 'lyric': '세븐틴은 표정관리 안돼', 'section': 'Chorus', 'line': 4}, {'member': ['Hoshi'], 'lyric': '내가 그래,\u2005yeah-yeah-yeah', 'section': 'Chorus', 'line': 5}, {'member': ['Hoshi'], 'lyric': '내가\u2005그래, yeah-yeah-yeah', 'section': 'Chorus', 'line': 6}, {'member': ['Hoshi'], 'lyric': '다른 소녀들\u2005너만큼은 안돼', 'sectio

In [444]:
with open('../json/final/' + album.replace(' ', '_') +'_final_discography.json', 'w') as file:
    json.dump(final_disco, file)

In [445]:
directory_path = Path('../json/final')
files = [p for p in directory_path.iterdir() if p.is_file()]

meta_data = []
for file in files:
    print(file)

    with open(file, 'r') as f:
        data = json.load(f)

    meta_data.append(data)

meta_data.sort(
    key=lambda x: date(
        x["release_date_components"]["year"],
        x["release_date_components"]["month"],
        x["release_date_components"]["day"]
    ),
    reverse=True
)

with open('../json/final_meta_data.json', 'w') as file:
        json.dump(meta_data, file, indent=4)

../json/final/YOU_MADE_MY_DAWN_final_discography.json
../json/final/Attacca_final_discography.json
../json/final/17_CARAT_final_discography.json
../json/final/You_Make_My_Day_final_discography.json
../json/final/;_[SEMICOLON]_final_discography.json
../json/final/Your_Choice_final_discography.json
../json/final/Heng:garæ_final_discography.json
../json/final/Al1_final_discography.json
../json/final/Boys_Be_final_discography.json
../json/final/SPILL_THE_FEELS_final_discography.json
../json/final/SEVENTEENTH_HEAVEN_final_discography.json
../json/final/Going_Seventeen_final_discography.json
../json/final/FML_final_discography.json
